In [11]:
include("../src/ODE.jl")

@ODEmodel (macro with 1 method)

In [16]:
# SIWR Cholera model

ode = @ODEmodel(
    S'(t) = mu - bi * S(t) * I(t) - bw * S(t) * W(t) - mu * S(t) + a * R(t),
    I'(t) = bw * S(t) * W(t) + bi * S(t) * I(t) - (gam + mu) * I(t),
    W'(t) = xi * (I(t) - W(t)),
    R'(t) = gam * I(t) - (mu + a) * R(t),
    y(t) = k * I(t)
)

ode_red = reduce_ode_mod_p(ode, 2^31 - 1)

@time ps = power_series_solution(
    ode_red,
    Dict(p => rand(1:10) for p in ode_red.parameters),
    Dict(x => rand(1:10) for x in ode_red.x_vars),
    Dict{typeof(ode_red.x_vars[1]), Array{Int64, 1}}(),
    1000
)

Summary of the model:
State variables: I, R, W, S
Parameter: bi, gam, mu, bw, k, xi, a
Inputs: 
Outputs: y
  8.459823 seconds (11.06 M allocations: 1021.696 MiB, 1.69% gc time)


Dict{AbstractAlgebra.Generic.MPoly{Nemo.gfp_elem},AbstractAlgebra.Generic.AbsSeries{Nemo.gfp_elem}} with 12 entries:
  1*xi  => 1 + O(t^1000)
  1*W   => 4 + 2147483644*t + 1073741877*t^2 + 357913825*t^3 + 268432209*t^4 + …
  1*a   => 8 + O(t^1000)
  1*R   => 9 + 2147483531*t + 1332*t^2 + 715820683*t^3 + 1073734262*t^4 + 12169…
  1*S   => 8 + 2147483557*t + 2147482916*t^2 + 19691*t^3 + 1968527291*t^4 + 534…
  1*bi  => 3 + O(t^1000)
  1*y   => 5 + 520*t + 2147482172*t^2 + 715762364*t^3 + 1968566001*t^4 + 197858…
  1*mu  => 6 + O(t^1000)
  1*k   => 5 + O(t^1000)
  1*gam => 10 + O(t^1000)
  1*bw  => 3 + O(t^1000)
  1*I   => 1 + 104*t + 2147483352*t^2 + 1431642661*t^3 + 1252706659*t^4 + 21137…

In [4]:
# Pharm

var_names = [
    "x_0", "x_1", "x_2", "x_3",
    "a1", "a2", "b1", "b2", "kc", "ka", "n"
]

R, (x_0, x_1, x_2, x_3, a1, a2, b1, b2, kc, ka, n) = PolynomialRing(F, var_names)

ode = ODE(
    Dict(
        x_0 => a1 * (x_1 - x_0) - (ka * n * x_0) // (kc * ka + kc * x_2 + ka * x_0),
        x_1 => a2 * (x_0 - x_1),
        x_2 => b1 * (x_3 - x_2) - (kc * n * x_2) // (kc * ka + kc * x_2 + ka * x_0),
        x_3 => b2 * (x_2 - x_3)
    ),
    []
)

params = [a1, a2, b1, b2, kc, ka, n]
states = [x_0, x_1, x_2, x_3]

@time ps = power_series_solution(
    ode,
    Dict(p => rand(1:10) for p in params),
    Dict(x => rand(1:10) for x in states),
    Dict(),
    1000
)

  6.987082 seconds (234.79 M allocations: 7.199 GiB, 12.32% gc time)


Dict{AbstractAlgebra.Generic.MPoly{Nemo.gfp_elem},AbstractAlgebra.Generic.AbsSeries{Nemo.gfp_elem}} with 4 entries:
  x_1 => 1+(16)*t+(2021161044)*t^2+(1380174874)*t^3+(1242606661)*t^4+(704047998…
  x_2 => 2+(1810623497)*t+(242947774)*t^2+(1856444455)*t^3+(1648496433)*t^4+(13…
  x_3 => 5+(2147483644)*t+(905311750)*t^2+(1926695655)*t^3+(2129920847)*t^4+(76…
  x_0 => 9+(2021161060)*t+(1943939708)*t^2+(1717904549)*t^3+(855243009)*t^4+(20…

In [7]:
# QWWC

var_names = [
    "x", "y", "z", "w",
    "a", "b", "c", "d", "e", "f"
]

R, (x, y, z, w, a, b, c, d, e, f) = PolynomialRing(F, var_names)

ode = ODE(
    Dict(
        x => a * (y - x) + y * z,
        y => b * (x + y) - x * z,
        z => - c * z - d * w + x * y,
        w => e * z - f * w + x * y
    ),
    []
)

params = [a, b, c, d, e, f]
states = [x, y, z, w]

@time ps = power_series_solution(
    ode,
    Dict(p => rand(1:10) for p in params),
    Dict(v => rand(1:10) for v in states),
    Dict(),
    1000
)

  6.144319 seconds (219.56 M allocations: 6.739 GiB, 13.89% gc time)


Dict{AbstractAlgebra.Generic.MPoly{Nemo.gfp_elem},AbstractAlgebra.Generic.AbsSeries{Nemo.gfp_elem}} with 4 entries:
  x => 5+(56)*t+(2147483053)*t^2+(1431657845)*t^3+(984261144)*t^4+(322104938)*t…
  y => 6+(2147483618)*t+(1073741728)*t^2+(1431657510)*t^3+(1431645202)*t^4+(173…
  z => 8+(2147483601)*t+(1073742104)*t^2+(1431653045)*t^3+(1610628003)*t^4+(644…
  w => 2+(22)*t+(1073741808)*t^2+(1431654011)*t^3+(894795952)*t^4+(1073732750)*…

In [8]:
# Model 41

var_names = [
    "S", "E", "A", "I", "J", "C",
    "b", "k", "g1", "g2", "alpha", "r", "q"
]
R, (S, E, A, I, J, C, b, k, g1, g2, alpha, r, q) = PolynomialRing(F, var_names)
ode = ODE(
    Dict(
        S => -b * S * (I + J + q * A),
        E => b * S * (I + J + q * A) - k * E,
        A => k * (1 - r) * E - g1 * A,
        I => k * r * E - (alpha + g1) * I,
        J => alpha * I - g2 * J,
        C => alpha * I
    ),
    []
)

params = [b, k, g1, g2, alpha, r, q]
states = [S, E, A, I, J, C]

@time ps = power_series_solution(
    ode,
    Dict(p => rand(1:10) for p in params),
    Dict(v => rand(1:10) for v in states),
    Dict(),
    1000
)

 11.865977 seconds (461.28 M allocations: 14.098 GiB, 12.67% gc time)


Dict{AbstractAlgebra.Generic.MPoly{Nemo.gfp_elem},AbstractAlgebra.Generic.AbsSeries{Nemo.gfp_elem}} with 6 entries:
  J => 9+(9)*t+(585)*t^2+(22098)*t^3+(2146695142)*t^4+(14281962)*t^5+(112898998…
  I => 5+(134)*t+(7626)*t^2+(715487257)*t^3+(723411859)*t^4+(759007530)*t^5+(15…
  A => 3+(2147483424)*t+(1073735099)*t^2+(1789847890)*t^3+(2052084480)*t^4+(152…
  E => 7+(552)*t+(2147456003)*t^2+(1432412160)*t^3+(1062385009)*t^4+(589421317)…
  C => 6+(45)*t+(603)*t^2+(22878)*t^3+(2146717240)*t^4+(13651158)*t^5+(11385112…
  S => 10+(2147483067)*t+(26540)*t^2+(1430936228)*t^3+(1800170125)*t^4+(1996644…